In [ ]:
# To convert a Python list into a PySpark DataFrame, use spark.createDataFrame() or convert it to an RDD first. Here's how:

from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("ListToDataFrame").getOrCreate()

# Python list
ls = [1, 2, 3, 5, 4, 7]

# Convert each element to a list (for row structure)
ls2 = [[i] for i in ls]

# Correct way: Pass ls2 directly
df = spark.createDataFrame(ls2, ["number"])

# Show DataFrame
df.show()

# Stop Spark session
spark.stop()


In [ ]:
# To convert the dictionary d = {1: "Tom", 2: "Brad", 3: "Joe"} into a PySpark DataFrame, follow these steps:

from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("DictToDataFrame").getOrCreate()

# Dictionary
d = {1: "Tom", 2: "Brad", 3: "Joe"}

# Convert dictionary to list of tuples
ls = list(d.items())  # [(1, 'Tom'), (2, 'Brad'), (3, 'Joe')]

# Create DataFrame with column names
df = spark.createDataFrame(ls, ["ID", "Name"])

# Show DataFrame
df.show()

# Stop Spark session
spark.stop()


In [ ]:
# To convert a list of tuples like ls = [(1, 3), (1, 4), (1, 5)] into a PySpark DataFrame, define column names during the conversion.

from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("TupleListToDataFrame").getOrCreate()

# List of tuples
ls = [(1, 3), (1, 4), (1, 5)]

# Convert list to DataFrame with column names
df = spark.createDataFrame(ls, ["col1", "col2"])

# Show DataFrame
df.show()

# Stop Spark session
spark.stop()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, concat_ws, when

# Initialize Spark session (not needed in Databricks notebooks)
spark = SparkSession.builder.appName("DatabricksETL").getOrCreate()

# Define file paths (modify according to your storage location)
parquet_path = "dbfs:/mnt/data/input.parquet"
json_path = "dbfs:/mnt/data/input.json"
delta_table_path = "dbfs:/mnt/data/output_delta"

# Read Parquet file
parquet_df = spark.read.parquet(parquet_path)

# Read JSON file
json_df = spark.read.json(json_path)

# ---- Step 1: Join both DataFrames on a common key (assuming 'id' exists in both) ----
df = parquet_df.join(json_df, on="id", how="inner")

# ---- Step 2: Add a new column combining name and address ----
df = df.withColumn("full_details", concat_ws(", ", col("name"), col("address")))

# ---- Step 3: Apply a transformation to modify the 'age' column ----
df = df.withColumn("age_group", when(col("age") < 18, "Minor")
                                  .when((col("age") >= 18) & (col("age") < 60), "Adult")
                                  .otherwise("Senior"))

# ---- Step 4: Add a static column to indicate the processing batch ----
df = df.withColumn("batch_date", lit("2025-03-30"))

# Write the transformed DataFrame as a Delta table
df.write.format("delta").mode("overwrite").save(delta_table_path)

# Optionally, create a Delta table if needed
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS processed_data
    USING DELTA
    LOCATION '{delta_table_path}'
""")

print("Data processing completed and saved as Delta table.")

'''

MERGE INTO target_table AS target
USING source_table AS source
ON target.id = source.id
WHEN MATCHED THEN
  UPDATE SET target.name = source.name, target.age = source.age
WHEN NOT MATCHED THEN
  INSERT (id, name, age) VALUES (source.id, source.name, source.age);

'''

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year

# Initialize Spark session
spark = SparkSession.builder.appName("SeniorDE_Interview").getOrCreate()

# File paths (modify as needed)
parquet_path = "dbfs:/mnt/data/customer_data.parquet"
csv_path = "dbfs:/mnt/data/transactions.csv"
delta_output_path = "dbfs:/mnt/data/output_delta"

# Read Parquet (Customer Data)
customer_df = spark.read.parquet(parquet_path)

# Read CSV (Transactions Data)
txn_df = spark.read.option("header", True).csv(csv_path)

# Convert txn_date to date type
txn_df = txn_df.withColumn("txn_date", col("txn_date").cast("date"))

# Join on customer_id
df = customer_df.join(txn_df, on="customer_id", how="inner")

# Filter transactions with amount > 1000
df = df.filter(col("amount") > 1000)

# Extract txn_year from txn_date
df = df.withColumn("txn_year", year(col("txn_date")))

# Write output as partitioned Delta table
df.write.format("delta").mode("overwrite").partitionBy("txn_year").save(delta_output_path)

print("Processing completed. Data written as Delta table.")

#############################################################
# Spark SQL based
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("SeniorDE_Interview").getOrCreate()

# File paths
parquet_path = "dbfs:/mnt/data/customer_data.parquet"
csv_path = "dbfs:/mnt/data/transactions.csv"
delta_output_path = "dbfs:/mnt/data/output_delta"

# Read Parquet (Customer Data) and create a temp view
customer_df = spark.read.parquet(parquet_path)
customer_df.createOrReplaceTempView("customer_data")

# Read CSV (Transactions Data) and create a temp view
txn_df = spark.read.option("header", True).csv(csv_path)
txn_df.createOrReplaceTempView("transactions_data")

# Convert txn_date to DATE format and create another view
spark.sql("""
    CREATE OR REPLACE TEMP VIEW transactions_cleaned AS
    SELECT txn_id, customer_id, amount, CAST(txn_date AS DATE) AS txn_date
    FROM transactions_data
""")

# SQL Query for Transformation
final_df = spark.sql("""
    WITH joined_data AS (
        SELECT c.customer_id, c.name, c.age, c.city, 
               t.txn_id, t.amount, t.txn_date,
               YEAR(t.txn_date) AS txn_year
        FROM customer_data c
        INNER JOIN transactions_cleaned t ON c.customer_id = t.customer_id
        WHERE t.amount > 1000
    )
    SELECT * FROM joined_data
""")

# Write the final DataFrame as a partitioned Delta table
final_df.write.format("delta").mode("overwrite").partitionBy("txn_year").save(delta_output_path)

print("Processing completed. Data written as a Delta table.")
